In [1]:
# CDS API

import cdsapi



# Libraries for working with multi-dimensional arrays

import xarray as xr

import pandas as pd

import numpy as np

import os

# Forecast verification metrics with xarray

import xskillscore as xs



# Date and calendar libraries

from dateutil.relativedelta import relativedelta

import calendar



# Libraries for plotting and geospatial data visualisation

from matplotlib import pyplot as plt

import cartopy.crs as ccrs

import cartopy.feature as cfeature



# Disable warnings for data download via API and matplotlib (do I need both???)

import warnings
import re
import os

warnings.filterwarnings('ignore')

In [2]:
file_path = "/home/mohamed/EHTPIII/MODELISATION/DATA/DATASET/IN/rea/era5_monthly_stmonth{_NOVEMBER_}_{1993}-{2016}.grib"
era5_1deg = xr.open_dataset(file_path, engine="cfgrib",drop_variables="tp")
era5_1deg.variables

skipping variable: paramId==228 shortName='tp'
Traceback (most recent call last):
  File "/home/mohamed/.local/lib/python3.12/site-packages/cfgrib/dataset.py", line 721, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/mohamed/.local/lib/python3.12/site-packages/cfgrib/dataset.py", line 639, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='time' value=Variable(dimensions=('time',), data=array([ 725846400,  728524800,  730944000,  733622400,  736214400,
        738892800,  741484800,  744163200,  746841600,  749433600,
        752112000,  754704000,  757382400,  760060800,  762480000,
        765158400,  767750400,  770428800,  773020800,  775699200,
        778377600,  780969600,  783648000,  786240000,  788918400,
        791596800,  794016000,  796694400,  799286400,  801964800,
        804556800,  807235200,  809913600,  812505600,  815184000,
        817776000,  820454400,  82313

Frozen({'number': <xarray.Variable ()> Size: 8B
[1 values with dtype=int64]
Attributes:
    long_name:      ensemble member numerical id
    units:          1
    standard_name:  realization, 'time': <xarray.IndexVariable 'time' (time: 288)> Size: 2kB
array(['1993-01-01T00:00:00.000000000', '1993-02-01T00:00:00.000000000',
       '1993-03-01T00:00:00.000000000', ..., '2016-10-01T00:00:00.000000000',
       '2016-11-01T00:00:00.000000000', '2016-12-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Attributes:
    long_name:      initial time of forecast
    standard_name:  forecast_reference_time, 'step': <xarray.Variable ()> Size: 8B
[1 values with dtype=timedelta64[ns]]
Attributes:
    long_name:      time since forecast_reference_time
    standard_name:  forecast_period, 'surface': <xarray.Variable ()> Size: 8B
[1 values with dtype=float64]
Attributes:
    long_name:  original GRIB coordinate for key: level(surface)
    units:      1, 'latitude': <xarray.IndexVariable 'latitude'

In [3]:
era5_1deg = era5_1deg.rename({'latitude':'lat','longitude':'lon','time':'start_date'}).swap_dims({'start_date':'valid_time'})
valid_time = pd.to_datetime(era5_1deg.valid_time)
valid_time_normalized = valid_time.normalize()
era5_1deg["valid_time"]=valid_time_normalized
era_anom=era5_1deg-era5_1deg.mean("valid_time",skipna=True)

In [4]:
DATAIN ="/home/mohamed/EHTPIII/MODELISATION/DATA/DATASET/IN/grib"
    
DATAOUT="/home/mohamed/EHTPIII/MODELISATION/DATA/DATASET/OUT"

grib_files = [file for file in os.listdir(DATAIN) if file.endswith('.grib')]
for file in grib_files:

    match = re.search(r"monthly_mean_(\d+)_", file)
    start=int(match.group(1))
    
    config = dict(
    
        list_vars = ['t2m', ],
    
        hcstarty = 1993,
    
        hcendy = 2016,
    
        start_month = start,
    
    )
    
    import os
    
    
    SCOREDIR = DATAOUT + '/SF/scores'
    
    PLOTSDIR = DATAOUT + f'/SF/plots/stmonth{config["start_month"]:02d}'
    
    
    
    for directory in [DATAIN, SCOREDIR, PLOTSDIR]:
    
        # Check if the directory exists
    
        if not os.path.exists(directory):
    
            # If it doesn't exist, create it
    
            os.makedirs(directory)
    
            print(f'Creating folder {directory}')
    st_dim_name = 'time' if not config.get('isLagged',False) else 'indexing_time'
    
    print('Reading HCST data from file')
    
    # available_files = ["ukmo_602", "meteo_france_8", "ecmwf_51", "eccc_3", "eccc_2", "dwd_21", "cmcc_35"]
    
    
    hcst_fname=DATAIN + f'/{file}'
    
    hcst_bname=file.split(".grib")[0]
    if "ecmwf" in hcst_bname:
        hcst = xr.open_dataset(hcst_fname,engine='cfgrib', backend_kwargs=dict(time_dims=('forecastMonth', st_dim_name),filter_by_keys={'paramId':167}))
    else:
        hcst = xr.open_dataset(hcst_fname,engine='cfgrib', backend_kwargs=dict(time_dims=('forecastMonth', st_dim_name),filter_by_keys={'paramId':167}))

    

    
    hcst = hcst.chunk({'forecastMonth':1, 'latitude':'auto', 'longitude':'auto'})  #force dask.array using chunks on leadtime, latitude and longitude coordinate
    
    hcst = hcst.rename({'latitude':'lat','longitude':'lon', st_dim_name:'start_date'})
    
    print ('Re-arranging time metadata in xr.Dataset object')
    
    # Add start_month to the xr.Dataset
    
    start_month = pd.to_datetime(hcst.start_date.values[0]).month
    
    hcst = hcst.assign_coords({'start_month':start_month})
    
    # Add valid_time to the xr.Dataset
    
    vt = xr.DataArray(dims=('start_date','forecastMonth'), coords={'forecastMonth':hcst.forecastMonth,'start_date':hcst.start_date})
    
    vt.data = [[pd.to_datetime(std)+relativedelta(months=fcmonth-1) for fcmonth in vt.forecastMonth.values] for std in vt.start_date.values]
    
    hcst = hcst.assign_coords(valid_time=vt)
    
    
    
    # CALCULATE 3-month AGGREGATIONS
    
    # NOTE rolling() assigns the label to the end of the N month period, so the first N-1 elements have NaN and can be dropped
    
    print('Computing 3-month aggregation')
    
    hcst_3m = hcst.rolling(forecastMonth=3,min_periods=1).mean(skipna=True)
    
    hcst_3m = hcst_3m.where(hcst_3m.forecastMonth>=3,drop=True)
    
    
    
    
    
    # CALCULATE ANOMALIES (and save to file)
    
    print('Computing anomalies 1m')
    
    hcmean = hcst.mean(['number','start_date'],skipna=True)
    
    anom = hcst - hcmean
    
    anom = anom.assign_attrs(reference_period='{hcstarty}-{hcendy}'.format(**config))
    
    
    
    hcst_2=hcst.assign_attrs(reference_period='{hcstarty}-{hcendy}'.format(**config))
    
    hcst_2_3m=hcst_2.rolling(forecastMonth=3,min_periods=1).mean(skipna=True)
    
    hcst_2_3m = hcst_2_3m.where(hcst_2_3m.forecastMonth>=3,drop=True)
    
    
    
    print('Computing anomalies 3m')
    
    hcmean_3m = hcst_3m.mean(['number','start_date'],skipna=True)
    
    anom_3m = hcst_3m - hcmean_3m
    
    anom_3m = anom_3m.assign_attrs(reference_period='{hcstarty}-{hcendy}'.format(**config))
    
    
    
    print('Saving anomalies 1m/3m to netCDF files')
    
    anom.to_netcdf(f'{DATAOUT}/{hcst_bname}.1m.anom.nc')
    
    hcst_2.to_netcdf(f'{DATAOUT}/{hcst_bname}.1m.hcst_2.nc')
    
    hcst_2_3m.to_netcdf(f'{DATAOUT}/{hcst_bname}.3m.hcst_2.nc')
    
    anom_3m.to_netcdf(f'{DATAOUT}/{hcst_bname}.3m.anom.nc')
    # We define a function to calculate the boundaries of forecast categories defined by quantiles
    
    def get_thresh(icat,quantiles,xrds,dims=['number','start_date']):
    
    
    
        if not all(elem in xrds.dims for elem in dims):           
    
            raise Exception('Some of the dimensions in {} is not present in the xr.Dataset {}'.format(dims,xrds)) 
    
        else:
    
            if icat == 0:
    
                xrds_lo = -np.inf
    
                xrds_hi = xrds.quantile(quantiles[icat],dim=dims)      
    
                
    
            elif icat == len(quantiles):
    
                xrds_lo = xrds.quantile(quantiles[icat-1],dim=dims)
    
                xrds_hi = np.inf
    
                
    
            else:
    
                xrds_lo = xrds.quantile(quantiles[icat-1],dim=dims)
    
                xrds_hi = xrds.quantile(quantiles[icat],dim=dims)
    
          
    
        return xrds_lo,xrds_hi
    
    print('Computing probabilities (tercile categories)')
    
    quantiles = [1/3., 2/3.]
    
    numcategories = len(quantiles)+1
    
    
    
    for aggr,h in [("1m",hcst), ("3m",hcst_3m)]:
    
        print(f'Computing tercile probabilities {aggr}')
    
    
    
        l_probs_hcst=list()
    
        for icat in range(numcategories):
    
            print(f'category={icat}')
    
            h_lo,h_hi = get_thresh(icat, quantiles, h)
    
            probh = np.logical_and(h>h_lo, h<=h_hi).sum('number')/float(h.dims['number'])
    
            # Instead of using the coordinate 'quantile' coming from the hindcast xr.Dataset
    
            # we will create a new coordinate called 'category'
    
            if 'quantile' in probh:
    
                probh = probh.drop('quantile')
    
            l_probs_hcst.append(probh.assign_coords({'category':icat}))
    
    
    
        print(f'Concatenating {aggr} tercile probs categories')
    
        probs = xr.concat(l_probs_hcst,dim='category')
    
        print(f'Saving {aggr} tercile probs netCDF files')
    
        probs.to_netcdf(f'{DATAOUT}/{hcst_bname}.{aggr}.tercile_probs.nc')
    
    # DATAIN2="/kaggle/input/reanalysis"
    # obs_fname="/kaggle/input/reanalysis/era5_monthly_stmonth_NOVEMBER__1993-2016.grib"
    # era5_1deg = xr.open_dataset(obs_fname, engine='cfgrib',drop_variables="t2m")
    # Renaming to match hindcast names

    
    
    
    # Assign 'forecastMonth' coordinate values
    
    fcmonths = [mm+1 if mm>=0 else mm+13 for mm in [t.month - config['start_month'] for t in pd.to_datetime(era5_1deg.valid_time.values)] ]
    
    era5_1deg = era5_1deg.assign_coords(forecastMonth=('valid_time',fcmonths))
    era_anom=era_anom.assign_coords(forecastMonth=('valid_time',fcmonths))
    
    # Drop obs values not needed (earlier than first start date) - this is useful to create well shaped 3-month aggregations from obs.
    
    era5_1deg = era5_1deg.where(era5_1deg.valid_time>=np.datetime64('{hcstarty}-{start_month:02d}-01'.format(**config)),drop=True)
    era_anom = era_anom.where(era_anom.valid_time>=np.datetime64('{hcstarty}-{start_month:02d}-01'.format(**config)),drop=True)

    
    era_anom_3m= era_anom.rolling(valid_time=3,min_periods=1).mean(skipna=True)
    era_anom_3m=era_anom_3m.where(era_anom_3m.forecastMonth>=3)
    
    
    # CALCULATE 3-month AGGREGATIONS
    
    # NOTE rolling() assigns the label to the end of the N month period
    
    print('Calculate observation 3-monthly aggregations')
    
    # NOTE care should be taken with the data available in the "obs" xr.Dataset so the rolling mean (over valid_time) is meaningful
    
    era5_1deg_3m = era5_1deg.rolling(valid_time=3,min_periods=1).mean(skipna=True)
    
    era5_1deg_3m = era5_1deg_3m.where(era5_1deg_3m.forecastMonth>=3)

    era_anom = era_anom.drop('forecastMonth')

    era_anom_3m = era_anom_3m.drop('forecastMonth')
    
    
    # As we don't need it anymore at this stage, we can safely remove
    
    # 'forecastMonth'
    
    era5_1deg = era5_1deg.drop('forecastMonth')
    
    era5_1deg_3m = era5_1deg_3m.drop('forecastMonth')
    
    from os.path import join
    
    # Loop over aggregations
    for aggr in ['1m', '3m']:
    
        if aggr == '1m':
            o = era5_1deg
            o_anom=era_anom
        elif aggr == '3m':
            o = era5_1deg_3m
            o_anom=era_anom_3m
        else:
            raise BaseException(f'Unknown aggregation {aggr}')
    
        print(f'Computing deterministic scores for {aggr}-aggregation')
    
        # Read anomalies file
        h_anom = xr.open_dataset(f'{DATAOUT}/{hcst_bname}.{aggr}.anom.nc')
        h=xr.open_dataset(f'{DATAOUT}/{hcst_bname}.{aggr}.hcst_2.nc')

        if "t2m" not in h.variables:
    # Rename the variable 'p167' to 't2m'
            h_anom = h_anom.rename({'p167': 't2m'})
            h = h.rename({'p167': 't2m'})
        # h=h.rename({'p167':'t2m'})

        # o=o.swap_dims({'time': 'valid_time'})
    
        is_fullensemble = 'number' in h_anom.dims
    
        l_corr = list()
        l_acc=list()
        l_corr_pval = list()
        l_acc_pval=list()
    
        for this_fcmonth in h.forecastMonth.values:
            print(f'forecastMonth={this_fcmonth}')
            thishcst_anom = h_anom.sel(forecastMonth=this_fcmonth).swap_dims({'start_date': 'valid_time'})
            thishcst = h.sel(forecastMonth=this_fcmonth).swap_dims({'start_date': 'valid_time'})
    
            thisobs_anom = o_anom.where(o_anom.valid_time == thishcst_anom.valid_time, drop=True)
            thisobs = o.where(o.valid_time == thishcst.valid_time, drop=True)
    
            # Align the forecast and observation data along all common dimensions
            thishcst_em, thisobs_aligned = xr.align(thishcst, thisobs, join='inner')
            thishcst_em_anom, thisobs_aligned_anom = xr.align(thishcst_anom, thisobs_anom, join='inner')
    
            # If it's a full ensemble, take the mean over the 'number' dimension
            thishcst_em = thishcst_em if not is_fullensemble else thishcst_em.mean('number',skipna=True)
            thishcst_em_anom = thishcst_em_anom if not is_fullensemble else thishcst_em_anom.mean('number',skipna=True)
    
            l_corr.append(xs.spearman_r(thishcst_em, thisobs_aligned, dim='valid_time'))
            l_acc.append(xs.pearson_r(thishcst_em_anom, thisobs_aligned_anom, dim='valid_time'))
            l_corr_pval.append(xs.spearman_r_p_value(thishcst_em, thisobs_aligned, dim='valid_time'))
            l_acc_pval.append(xs.pearson_r_p_value(thishcst_em_anom, thisobs_aligned_anom, dim='valid_time'))

        print(f'Concatenating (by fcmonth) correlation for {aggr}-aggregation')
        corr = xr.concat(l_corr, dim='forecastMonth')
        rsquared=corr ** 2
        corr_pval = xr.concat(l_corr_pval, dim='forecastMonth')
        acc=xr.concat(l_acc,dim='forecastMonth')
        acc_pval=xr.concat(l_acc_pval,dim='forecastMonth')
    
        print(f'Saving to netCDF file correlation for {aggr}-aggregation')
        corr.to_netcdf(f'{SCOREDIR}/{hcst_bname}.{aggr}.corr.nc')
        corr_pval.to_netcdf(f'{SCOREDIR}/{hcst_bname}.{aggr}.corr_pval.nc')
        rsquared.to_netcdf(f'{SCOREDIR}/{hcst_bname}.{aggr}.rsquared.nc')
        
        acc.to_netcdf(f'{SCOREDIR}/{hcst_bname}.{aggr}.acc.nc')
        acc_pval.to_netcdf(f'{SCOREDIR}/{hcst_bname}.{aggr}.acc_pval.nc')



    from os.path import join
# Loop over aggregations
    for aggr in ['1m','3m']:

        if aggr == '1m':
            o = era5_1deg
        elif aggr == '3m':
            o = era5_1deg_3m
        else:
            raise BaseException(f'Unknown aggregation {aggr}')
    
        print(f'Computing deterministic scores for {aggr}-aggregation')
    
        # Read anomalies file
        h = xr.open_dataset(f'{DATAOUT}/{hcst_bname}.{aggr}.hcst_2.nc')
        if "t2m" not in h.variables:
            h=h.rename({'p167':'t2m'})
        is_fullensemble = 'number' in h.dims
    
        l_rmse = list()
    
        for this_fcmonth in h.forecastMonth.values:
            print(f'forecastMonth={this_fcmonth}')
            thishcst = h.sel(forecastMonth=this_fcmonth).swap_dims({'start_date': 'valid_time'})
            thisobs = o.where(o.valid_time == thishcst.valid_time, drop=True)
    
            # Align the forecast and observation data along all common dimensions
            thishcst_em, thisobs_aligned = xr.align(thishcst, thisobs, join='inner')
    
            # If it's a full ensemble, take the mean over the 'number' dimension
            thishcst_em = thishcst_em if not is_fullensemble else thishcst_em.mean('number',skipna=True)
    
            l_rmse.append(xs.rmse(thishcst_em, thisobs_aligned, dim='valid_time'))
    
        print(f'Concatenating (by fcmonth) correlation for {aggr}-aggregation')
        rmse = xr.concat(l_rmse, dim='forecastMonth')
    
        print(f'Saving to netCDF file correlation for {aggr}-aggregation')
        rmse.to_netcdf(f'{SCOREDIR}/{hcst_bname}.{aggr}.rmse.nc')


    
    
    from os.path import join
    
    
    
    # Loop over aggregations
    
    for aggr in ['1m', '3m']:

        if aggr == '1m':
    
            o = era5_1deg
    
        elif aggr == '3m':
    
            o = era5_1deg_3m
    
        else:
    
            raise BaseException(f'Unknown aggregation {aggr}')
    
        print(f'Computing deterministic scores for {aggr}-aggregation')
    
        # Read hindcast probabilities file
    
        probs_hcst = xr.open_dataset(f'{DATAOUT}/{hcst_bname}.{aggr}.tercile_probs.nc')

        if "t2m" not in probs_hcst.variables:
            probs_hcst=probs_hcst.rename({"p167":"t2m"})
    
        
    
        l_roc = list()
    
        l_rps = list()
    
        l_rocss = list()
    
        l_bs = list()
    
        l_rela=list()
    
    
        for this_fcmonth in probs_hcst.forecastMonth.values:
    
            print(f'forecastMonth={this_fcmonth}')
    
            thishcst = probs_hcst.sel(forecastMonth=this_fcmonth).swap_dims({'start_date': 'valid_time'})
    
            # CALCULATE probabilities from observations
    
            print('We need to calculate probabilities (tercile categories) from observations')
    
            l_probs_obs = list()
    
            # Align both forecast and observation data on 'valid_time'
    
            thiso = o.where(o.valid_time == thishcst.valid_time, drop=True)
    
            thishcst_aligned, thiso_aligned = xr.align(thishcst, thiso, join='inner')
    
            for icat in range(3):
    
                # Compute category thresholds and probabilities for observations
    
                o_lo, o_hi = get_thresh(icat, quantiles, thiso_aligned, dims=['valid_time'])
    
                probo = 1. * np.logical_and(thiso_aligned > o_lo, thiso_aligned <= o_hi)
    
                if 'quantile' in probo:
    
                    probo = probo.drop('quantile')
    
                l_probs_obs.append(probo.assign_coords({'category': icat}))
    
            thisobs = xr.concat(l_probs_obs, dim='category')
    
            # Now we can calculate the probabilistic (tercile categories) scores
    
            print('Now we can calculate the probabilistic (tercile categories) scores')
    
            thisroc = xr.Dataset()
    
            thisrps = xr.Dataset()
    
            thisrocss = xr.Dataset()
    
            thisbs = xr.Dataset()
    
            this_rela=xr.Dataset()
    
    
    
            for var in thishcst_aligned.data_vars:
    
                thisroc[var] = xs.roc(thisobs[var], thishcst_aligned[var], dim='valid_time', bin_edges=np.linspace(0,1,101))
    
    
    
                thisrps[var] = xs.rps(thisobs[var], thishcst_aligned[var], dim='valid_time', category_edges=None, input_distributions='p')
    
                
    
                thisobs_binary = thisobs[var].astype(bool)
                forecast_probs = thishcst_aligned[var].clip(0, 1) 
                this_rela[var] = xs.reliability(observations=thisobs_binary, forecasts=forecast_probs, dim="valid_time")
    
    
    
                thisrocss[var] = (thisroc[var] - 0.5) / (1. - 0.5)
    
    
    
                bscat = list()
    
                for cat in thisobs[var].category:
    
                    print(f'compute BS ({var}) for cat={cat.values}' )
    
                    thisobscat = thisobs[var].sel(category=cat)
    
                    thishcstcat = thishcst_aligned[var].sel(category=cat)
    
                    bscat.append(xs.brier_score(thisobscat, thishcstcat, dim='valid_time'))
    
    
    
                thisbs[var] = xr.concat(bscat, dim='category')
    
    
    
            l_roc.append(thisroc)
    
            l_rps.append(thisrps)
    
            l_rocss.append(thisrocss)
    
            l_bs.append(thisbs)
            
            l_rela.append(this_rela)
    
    
    
        print('concat roc')
    
        roc = xr.concat(l_roc, dim='forecastMonth')
    
        print('concat rps')
    
        rps = xr.concat(l_rps, dim='forecastMonth')
    
        print('concat rocss')
    
        rocss = xr.concat(l_rocss, dim='forecastMonth')
    
        print('concat bs')
    
        bs = xr.concat(l_bs, dim='forecastMonth')
    
        print('concat rela')
    
        rela=xr.concat(l_rela,dim='forecastMonth')
    
    
    
        print('writing to netcdf rps')
    
        rps.to_netcdf(f'{SCOREDIR}/{hcst_bname}.{aggr}.rps.nc')
    
        print('writing to netcdf bs')
    
        bs.to_netcdf(f'{SCOREDIR}/{hcst_bname}.{aggr}.bs.nc')
    
        print('writing to netcdf roc')
    
        roc.to_netcdf(f'{SCOREDIR}/{hcst_bname}.{aggr}.roc.nc')
    
        print('writing to netcdf rocss')
    
        rocss.to_netcdf(f'{SCOREDIR}/{hcst_bname}.{aggr}.rocss.nc')
    
        print('writing to netcdf rela')
    
        rela.to_netcdf(f'{SCOREDIR}/{hcst_bname}.{aggr}.rela.nc')

Reading HCST data from file
Re-arranging time metadata in xr.Dataset object
Computing 3-month aggregation
Computing anomalies 1m
Computing anomalies 3m
Saving anomalies 1m/3m to netCDF files
Computing probabilities (tercile categories)
Computing tercile probabilities 1m
category=0
category=1
category=2
Concatenating 1m tercile probs categories
Saving 1m tercile probs netCDF files
Computing tercile probabilities 3m
category=0
category=1
category=2
Concatenating 3m tercile probs categories
Saving 3m tercile probs netCDF files
Calculate observation 3-monthly aggregations
Computing deterministic scores for 1m-aggregation
forecastMonth=2
forecastMonth=3
forecastMonth=4
Concatenating (by fcmonth) correlation for 1m-aggregation
Saving to netCDF file correlation for 1m-aggregation
Computing deterministic scores for 3m-aggregation
forecastMonth=3
forecastMonth=4
Concatenating (by fcmonth) correlation for 3m-aggregation
Saving to netCDF file correlation for 3m-aggregation
Computing deterministic